In [1]:
{-# LANGUAGE TypeOperators, FlexibleContexts, TypeFamilies
 , UnicodeSyntax, ScopedTypeVariables, NoMonomorphismRestriction #-}
import Prelude ()
import Data.Manifold.TreeCover
import Data.Manifold.Shade
import Data.Random
import Data.Random.Manifold
import Data.Manifold
import Data.Manifold.Web
import Data.Manifold.DifferentialEquation
import Math.LinearMap.Category
import Data.VectorSpace
import Data.Function.Affine
import Data.Basis (Basis)
import Linear(V2(..), ex, ey, _x, _y)
import Data.Semigroup
import Data.List.NonEmpty (NonEmpty(..))
import qualified Data.Foldable as Hask
import qualified Control.Monad as Hask
import qualified Control.Comonad.Cofree as Hask
import Control.Lens
:opt no-lint
import Control.Category.Constrained.Prelude
import Control.Arrow.Constrained
test x = x `seq` putStrLn "ok"

From [diagrams](http://projects.haskell.org/diagrams):

In [2]:
import Diagrams.Prelude (p2, circle, (&), (^&), moveTo, opacity, fromVertices, Point(P))

In [3]:
type X = ℝ
type T = ℝ
type U = ℝ
type Ðx'U = ℝ
type Ðt'U = ℝ
type Ðx'Ðt'U = ℝ
type Ðx²'U = ℝ
type x × y = ℝ²
type HeatFlow = (U, Ðx'U)
et = ey :: Basis ℝ²
_t = _y :: Lens' (X × T) T
_u = _1 :: Lens' (U, Ðx'U) U
_ðx'U = _2 :: Lens' (U, Ðx'U) Ðx'U

From [dynamic-plot](http://hackage.haskell.org/package/dynamic-plot):

In [4]:
import Graphics.Dynamic.Plot.R2
import Data.Colour.Names
import Data.Colour.Manifold

colourscheme :: Shade' ℝ -> Shade (Colour ℝ)
colourscheme (Shade' u du) = interp (Shade u $ dualNorm du :: Shade ℝ)
 where Just interp = rangeOnGeodesic darkblue orange
 
prettyWebPlot :: PointsWeb ℝ² y -> DynamicPlottable
prettyWebPlot w = plot [ diagramPlot . opacity 0.5 $ fromVertices [P r₁, P r₂]
                       | ((r₁@(V2 x₁ y₁),_),(r₂@(V2 x₂ y₂),_)) <- edg ]
 where edg = webEdges w
 
colourscheme_heat :: Shade' HeatFlow -> Shade (Colour ℝ)
colourscheme_heat = f `seq` \(Shade' y e) -> f (Shade y $ dualNorm e)
 where Just (f :: Shade HeatFlow -> Shade (Colour ℝ))
          = rangeWithinVertices ((0,0), neutral)
                              [ ((1,0), orange)
                              , ((0,2), seagreen) ]
       Just neutral = toInterior (dimgrey :: Colour ℝ)
test colourscheme_heat

ok

In [5]:
initState_heat :: X -> U
initState_heat x = u
 where u = tanh (s * (1 - x^2))
       s = 1

plotWindow [continFnPlot initState_heat, xInterval (-3,3)]

GraphWindowSpecR2{lBound=-4.0, rBound=4.0, bBound=-1.2868473487266554, tBound=1.007986483048605, xResolution=640, yResolution=480}

In [6]:
α = 0.01

heatEqn :: DifferentialEqn (X × T) Ðx²'U (U, Ðx'U)
heatEqn (Shade (V2 x t, _) _) = LocalDifferentialEqn {
     _predictDerivatives = \shj
       -> return $ embedShade (lensEmbedding (α ∂_u/∂_t)) shj
   , _rescanDerivatives = \shj shy
       -> ( return $
                        (if abs t>0.1 then projectShade (lensEmbedding _u) shy
                                      else initState_heat x|±|[1e-6] )
                      ✠ projectShade (lensEmbedding (1 ∂_u/∂_x)) shj
                   , return $ projectShade (lensEmbedding (α ∂_u/∂_t)) shj )
   }

In [7]:
startSt_heat = fromWebNodes euclideanMetric
          [ (V2 x t, zeroV|±|[(1,0), (0,10)])
          | x<-[xl,xl+δx₀ .. xr], t<-[tStart, tStart+δt₀ .. tEnd]]
 where δx₀ = 0.125
       δt₀ = 0.125
       (xl,xr) = (0,2)
       (tStart, tEnd) = (-0.125, 1)

tfs_heat = iterateFilterDEqn_static (indicateInconsistencies intersectShade's) id heatEqn startSt_heat
forM_ [Hask.toList tfs_heat ]
  $ \tfs ->
    plotWindow
       [ prettyWebPlot $ head tfs
       , plotLatest [ plot (fmap colourscheme_heat tfi)
                       & legendName ("i = "++show i)
                    | (i,tfi) <- zip [0..] tfs ] ]
Hask.mapM_ (\st -> mapM_ (\((p,y),(q,z)) -> putStrLn $
                 show p ++ ":\t" ++ prettyShowShade' y
            ++ "\n" ++ show q ++ ":\t" ++ prettyShowShade' z) (webEdges st) >> print())
    $ take 0 (Hask.toList tfs_heat)

In [8]:
import Control.Monad.Trans.Except
findErr :: (Hask.Monad m, Hask.Foldable m) => Hask.Cofree m a -> m ()
findErr (a Hask.:< q) = case Hask.toList q of
        [] -> const () <$> q
        l  -> foldr1 (>>) $ findErr<$>l

case _runExcept $ findErr tfs_heat of
 Left (PropagationInconsistency pro apr) -> do
      putStrLn $ "apriori: "++show apr
      forM_ pro $ \(o,m) -> putStrLn $ show o ++ "\t-> " ++ show m
      plotWindow $ (plot apr & legendName "apriori")
                 : [plot m & legendName ("from "++show o) | (o,m)<-pro]


Found hole ‘_runExcept’ with type: f0 (Except (PropagationInconsistency (X × T) (Shade' (U, Ðx'U))) ()) (Either (PropagationInconsistency a0 a1) t0)
Where: ‘f0’ is an ambiguous type variable
       ‘t0’ is an ambiguous type variable
       ‘a0’ is an ambiguous type variable
       ‘a1’ is an ambiguous type variable
Relevant bindings include it ∷ IO Graphics.Dynamic.Plot.R2.GraphWindowSpec (bound at <interactive>:1:1)
In the first argument of ‘($)’, namely ‘_runExcept’
In the expression: _runExcept $ findErr tfs_heat
In the expression:
  case _runExcept $ findErr tfs_heat of {
    Left (PropagationInconsistency pro apr)
      -> do { putStrLn $ "apriori: " ++ show apr;
              forM_ pro $ \ (o, m) -> putStrLn $ show o ++ "\t-> " ++ show m;
              plotWindow $ (plot apr & legendName "apriori") : [plot m & legendName ("from " ++ show o) | (o, m) ← pro] } }

![Initial state from which to start refining solution of the heat PDE](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/PDE-solution-filter/HeatEqn-InitialState.png)

In [9]:
length tfs_heat

20